In [79]:
!pip install optuna

In [80]:
import pandas as pd
from xgboost import XGBClassifier
import numpy as np
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import cross_validate
import optuna
import warnings
warnings.filterwarnings('ignore')

In [81]:
international_matches = pd.read_csv('/content/international_matches.csv')
world_cup_2022 = pd.read_csv('/content/2022_world_cup_matches.csv')
world_cup_2022.drop('Year', axis = 1, inplace = True)
world_cup_teams = pd.read_csv('/content/2022_world_cup_matches.csv')
print("Total rows before filtering:", international_matches.shape[0])

Total rows before filtering: 17769


In [82]:
world_cup_2022.head()

,ID,Date,Stage,Home Team,Away Team,Host Team
0,1,2022/11/20,Group stage,Qatar,Ecuador,True
1,2,2022/11/21,Group stage,Senegal,Netherlands,False
2,3,2022/11/21,Group stage,England,Iran,False
3,4,2022/11/21,Group stage,United States,Wales,False
4,5,2022/11/22,Group stage,France,Australia,False


In [83]:
world_cup_2022['Tournament Weight'] = 1
world_cup_2022['Recent Weight'] = 1

In [84]:
international_matches.isnull().sum()

,0
ID,0
Tournament,0
Date,0
Home Team,0
Home Goals,0
Away Goals,0
Away Team,0
Winning Team,4170
Losing Team,4170
Win Conditions,17568


In [85]:
international_matches.drop('Win Conditions', axis = 1, inplace = True)
international_matches.head()

,ID,Tournament,Date,Home Team,Home Goals,Away Goals,Away Team,Winning Team,Losing Team,Home Stadium
0,1,Friendly,1872-11-30,Scotland,0,0,England,NaN,NaN,True
1,2,Friendly,1873-03-08,England,4,2,Scotland,England,Scotland,True
2,3,Friendly,1874-03-07,Scotland,2,1,England,Scotland,England,True
3,4,Friendly,1875-03-06,England,2,2,Scotland,NaN,NaN,True
4,5,Friendly,1876-03-04,Scotland,3,0,England,Scotland,England,True


In [86]:
international_matches.isnull().sum()
international_matches.shape

(17769, 10)

In [87]:
international_matches['Year'] = international_matches['Date'].str[:4].astype(int)
international_matches.shape

(17769, 11)

In [88]:
international_matches.shape

(17769, 11)

In [89]:
international_matches.drop('Date', axis = 1, inplace = True)

In [90]:
international_matches.shape

(17769, 10)

In [91]:
international_matches = international_matches.dropna()

In [92]:
international_matches.shape

(13599, 10)

In [93]:
international_matches.isnull().sum()

,0
ID,0
Tournament,0
Home Team,0
Home Goals,0
Away Goals,0
Away Team,0
Winning Team,0
Losing Team,0
Home Stadium,0
Year,0


In [94]:
international_matches = international_matches[international_matches['Year'] >= 1950]

In [95]:
international_matches.shape

(12086, 10)

In [96]:
world_cup_2022.head()

,ID,Date,Stage,Home Team,Away Team,Host Team,Tournament Weight,Recent Weight
0,1,2022/11/20,Group stage,Qatar,Ecuador,True,1,1
1,2,2022/11/21,Group stage,Senegal,Netherlands,False,1,1
2,3,2022/11/21,Group stage,England,Iran,False,1,1
3,4,2022/11/21,Group stage,United States,Wales,False,1,1
4,5,2022/11/22,Group stage,France,Australia,False,1,1


In [97]:
fifa_teams = {
    'Afghanistan', 'Albania', 'Algeria', 'Andorra', 'Angola', 'Antigua and Barbuda', 'Argentina', 'Armenia', 'Aruba',
    'Australia', 'Austria', 'Azerbaijan', 'Bahamas', 'Bahrain', 'Bangladesh', 'Barbados', 'Belarus', 'Belgium',
    'Belize', 'Benin', 'Bermuda', 'Bhutan', 'Bolivia', 'Bosnia and Herzegovina', 'Botswana', 'Brazil', 'British Virgin Islands',
    'Brunei Darussalam', 'Bulgaria', 'Burkina Faso', 'Burundi', 'Cabo Verde', 'Cambodia', 'Cameroon', 'Canada',
    'Cayman Islands', 'Central African Republic', 'Chad', 'Chile', 'China PR', 'Chinese Taipei', 'Colombia', 'Comoros',
    'Congo', 'Cook Islands', 'Costa Rica', "Côte d'Ivoire", 'Croatia', 'Cuba', 'Curaçao', 'Cyprus', 'Czech Republic',
    'Democratic Republic of the Congo', 'Denmark', 'Djibouti', 'Dominica', 'Dominican Republic', 'Ecuador', 'Egypt',
    'El Salvador', 'England', 'Equatorial Guinea', 'Eritrea', 'Estonia', 'Eswatini', 'Ethiopia', 'Faroe Islands',
    'Fiji', 'Finland', 'France', 'Gabon', 'Gambia', 'Georgia', 'Germany', 'Ghana', 'Gibraltar', 'Greece', 'Grenada',
    'Guam', 'Guatemala', 'Guinea', 'Guinea-Bissau', 'Guyana', 'Haiti', 'Honduras', 'Hong Kong', 'Hungary', 'Iceland',
    'India', 'Indonesia', 'Iran', 'Iraq', 'Ireland', 'Israel', 'Italy', 'Jamaica', 'Japan', 'Jordan', 'Kazakhstan',
    'Kenya', 'Korea DPR', 'Korea Republic', 'Kosovo', 'Kuwait', 'Kyrgyz Republic', 'Lao', 'Latvia', 'Lebanon',
    'Lesotho', 'Liberia', 'Libya', 'Liechtenstein', 'Lithuania', 'Luxembourg', 'Macau', 'Madagascar', 'Malawi',
    'Malaysia', 'Maldives', 'Mali', 'Malta', 'Mauritania', 'Mauritius', 'Mexico', 'Moldova', 'Mongolia', 'Montenegro',
    'Montserrat', 'Morocco', 'Mozambique', 'Myanmar', 'Namibia', 'Nepal', 'Netherlands', 'New Caledonia',
    'New Zealand', 'Nicaragua', 'Niger', 'Nigeria', 'North Macedonia', 'Northern Ireland', 'Norway', 'Oman',
    'Pakistan', 'Palestine', 'Panama', 'Papua New Guinea', 'Paraguay', 'Peru', 'Philippines', 'Poland', 'Portugal',
    'Puerto Rico', 'Qatar', 'Romania', 'Russia', 'Rwanda', 'Saint Kitts and Nevis', 'Saint Lucia',
    'Saint Vincent and the Grenadines', 'Samoa', 'San Marino', 'Sao Tome and Principe', 'Saudi Arabia', 'Scotland',
    'Senegal', 'Serbia', 'Seychelles', 'Sierra Leone', 'Singapore', 'Slovakia', 'Slovenia', 'Solomon Islands',
    'Somalia', 'South Africa', 'South Sudan', 'Spain', 'Sri Lanka', 'Sudan', 'Suriname', 'Sweden', 'Switzerland',
    'Syria', 'Tajikistan', 'Tanzania', 'Thailand', 'Timor-Leste', 'Togo', 'Tonga', 'Trinidad and Tobago', 'Tunisia',
    'Turkey', 'Turkmenistan', 'Turks and Caicos Islands', 'Uganda', 'Ukraine', 'United Arab Emirates', 'United States',
    'Uruguay', 'Uzbekistan', 'Vanuatu', 'Venezuela', 'Vietnam', 'Wales', 'Yemen', 'Zambia', 'Zimbabwe'
}

international_matches = international_matches[
    international_matches['Home Team'].isin(fifa_teams) & international_matches['Away Team'].isin(fifa_teams)
]

international_matches.head()

,ID,Tournament,Home Team,Home Goals,Away Goals,Away Team,Winning Team,Losing Team,Home Stadium,Year
1810,1811,Friendly,Croatia,4,1,Serbia,Croatia,Serbia,True,1950
1811,1812,Friendly,Italy,3,1,Belgium,Italy,Belgium,True,1950
1815,1816,Copa Chevallier Boutell,Argentina,4,0,Paraguay,Argentina,Paraguay,True,1950
1816,1817,FIFA World Cup qualification,Spain,5,1,Portugal,Spain,Portugal,True,1950
1817,1818,Friendly,Chile,1,5,Uruguay,Uruguay,Chile,True,1950


In [98]:
international_matches.shape

(10640, 10)

In [99]:
def result(row):
    if row['Home Goals'] > row['Away Goals']:
        return 'Home'
    elif row['Home Goals'] < row['Away Goals']:
        return 'Away'
    else:
        return 'Draw'

international_matches['Result'] = international_matches.apply(result, axis=1)

In [100]:
international_matches.head()

,ID,Tournament,Home Team,Home Goals,Away Goals,Away Team,Winning Team,Losing Team,Home Stadium,Year,Result
1810,1811,Friendly,Croatia,4,1,Serbia,Croatia,Serbia,True,1950,Home
1811,1812,Friendly,Italy,3,1,Belgium,Italy,Belgium,True,1950,Home
1815,1816,Copa Chevallier Boutell,Argentina,4,0,Paraguay,Argentina,Paraguay,True,1950,Home
1816,1817,FIFA World Cup qualification,Spain,5,1,Portugal,Spain,Portugal,True,1950,Home
1817,1818,Friendly,Chile,1,5,Uruguay,Uruguay,Chile,True,1950,Away


In [101]:
# Combine all unique team names from both columns
all_teams = pd.concat([international_matches['Home Team'], international_matches['Away Team']]).unique()

# Fit LabelEncoder on all team names
le = LabelEncoder()
le.fit(all_teams)

# Save original team names for later use
international_matches['Home Team Name'] = international_matches['Home Team']
international_matches['Away Team Name'] = international_matches['Away Team']

# Encode both columns
international_matches['Home Team'] = le.transform(international_matches['Home Team'])
international_matches['Away Team'] = le.transform(international_matches['Away Team'])

In [102]:
international_matches.head()

,ID,Tournament,Home Team,Home Goals,Away Goals,Away Team,Winning Team,Losing Team,Home Stadium,Year,Result,Home Team Name,Away Team Name
1810,1811,Friendly,41,4,1,152,Croatia,Serbia,True,1950,Home,Croatia,Serbia
1811,1812,Friendly,86,3,1,16,Italy,Belgium,True,1950,Home,Italy,Belgium
1815,1816,Copa Chevallier Boutell,6,4,0,134,Argentina,Paraguay,True,1950,Home,Argentina,Paraguay
1816,1817,FIFA World Cup qualification,161,5,1,138,Spain,Portugal,True,1950,Home,Spain,Portugal
1817,1818,Friendly,34,1,5,182,Uruguay,Chile,True,1950,Away,Chile,Uruguay


In [103]:
elo_ratings = {}
def get_elo(team):
    return elo_ratings.get(team, 1500)

def update_elo(home, away, result, k=40):
    R_home = get_elo(home)
    R_away = get_elo(away)

    E_home = 1 / (1 + 10 ** ((R_away - R_home) / 400))
    E_away = 1 - E_home

    if result == 'Home':
        S_home, S_away = 1, 0
    elif result == 'Away':
        S_home, S_away = 0, 1
    else:
        S_home, S_away = 0.5, 0.5

    new_R_home = R_home + k * (S_home - E_home)
    new_R_away = R_away + k * (S_away - E_away)

    elo_ratings[home] = new_R_home
    elo_ratings[away] = new_R_away

elo_history_home = []
elo_history_away = []

for _, row in international_matches.iterrows():
    home_team = row['Home Team Name']
    away_team = row['Away Team Name']
    result_val = row['Result']

    elo_history_home.append(get_elo(home_team))
    elo_history_away.append(get_elo(away_team))
    update_elo(home_team, away_team, result_val)

international_matches['Home ELO'] = elo_history_home
international_matches['Away ELO'] = elo_history_away

In [104]:
international_matches.head()

,ID,Tournament,Home Team,Home Goals,Away Goals,Away Team,Winning Team,Losing Team,Home Stadium,Year,Result,Home Team Name,Away Team Name,Home ELO,Away ELO
1810,1811,Friendly,41,4,1,152,Croatia,Serbia,True,1950,Home,Croatia,Serbia,1500.0,1500.0
1811,1812,Friendly,86,3,1,16,Italy,Belgium,True,1950,Home,Italy,Belgium,1500.0,1500.0
1815,1816,Copa Chevallier Boutell,6,4,0,134,Argentina,Paraguay,True,1950,Home,Argentina,Paraguay,1500.0,1500.0
1816,1817,FIFA World Cup qualification,161,5,1,138,Spain,Portugal,True,1950,Home,Spain,Portugal,1500.0,1500.0
1817,1818,Friendly,34,1,5,182,Uruguay,Chile,True,1950,Away,Chile,Uruguay,1500.0,1500.0


In [105]:
international_matches['Home ELO'].unique()

array([1500.        , 1480.        , 1478.66012601, ..., 1919.38717377,
       1817.48682359, 1589.51643474])

In [106]:
def compute_win_rate(df, team_col, result_col):
    win_rate = []
    team_history = {}
    for idx, row in df.iterrows():
        team = row[team_col]
        results = team_history.get(team, [])
        if len(results) == 0:
            win_rate.append(0.5)
        else:
            win_rate.append(sum(results[-5:]) / min(5, len(results)))

        win = int(row[result_col] == 'Home') if team_col == 'Home Team' else int(row[result_col] == 'Away')
        results.append(win)
        team_history[team] = results
    return win_rate

international_matches['Home Win Rate'] = compute_win_rate(international_matches, 'Home Team', 'Result')
international_matches['Away Win Rate'] = compute_win_rate(international_matches, 'Away Team', 'Result')

In [107]:
important_tournaments = {
    'FIFA World Cup': 1.0,
    'UEFA Euro': 0.9,
    'Copa America': 0.9,
    'Confederations Cup': 0.8,
    'Friendly': 0.3
}

# Ensure 'Tournament' column is a string
international_matches['Tournament'] = international_matches['Tournament'].astype(str)

# Assign weights — default to 0.6 for unknown tournaments
international_matches['Tournament Weight'] = international_matches['Tournament'].apply(
    lambda x: important_tournaments.get(x, 0.6)
)

In [108]:
recent_weight = (international_matches['Year'] - international_matches['Year'].min()) / \
                (international_matches['Year'].max() - international_matches['Year'].min())

international_matches['Recent Weight'] = recent_weight

In [109]:
international_matches.shape

(10640, 19)

In [110]:
international_matches.head()

,ID,Tournament,Home Team,Home Goals,Away Goals,Away Team,Winning Team,Losing Team,Home Stadium,Year,Result,Home Team Name,Away Team Name,Home ELO,Away ELO,Home Win Rate,Away Win Rate,Tournament Weight,Recent Weight
1810,1811,Friendly,41,4,1,152,Croatia,Serbia,True,1950,Home,Croatia,Serbia,1500.0,1500.0,0.5,0.5,0.3,0.0
1811,1812,Friendly,86,3,1,16,Italy,Belgium,True,1950,Home,Italy,Belgium,1500.0,1500.0,0.5,0.5,0.3,0.0
1815,1816,Copa Chevallier Boutell,6,4,0,134,Argentina,Paraguay,True,1950,Home,Argentina,Paraguay,1500.0,1500.0,0.5,0.5,0.6,0.0
1816,1817,FIFA World Cup qualification,161,5,1,138,Spain,Portugal,True,1950,Home,Spain,Portugal,1500.0,1500.0,0.5,0.5,0.6,0.0
1817,1818,Friendly,34,1,5,182,Uruguay,Chile,True,1950,Away,Chile,Uruguay,1500.0,1500.0,0.5,0.5,0.3,0.0


In [111]:
international_matches['Recent Weight'].unique()

array([0.        , 0.01388889, 0.02777778, 0.04166667, 0.05555556,
       0.06944444, 0.08333333, 0.09722222, 0.11111111, 0.125     ,
       0.13888889, 0.15277778, 0.16666667, 0.18055556, 0.19444444,
       0.20833333, 0.22222222, 0.23611111, 0.25      , 0.26388889,
       0.27777778, 0.29166667, 0.30555556, 0.31944444, 0.33333333,
       0.34722222, 0.36111111, 0.375     , 0.38888889, 0.40277778,
       0.41666667, 0.43055556, 0.44444444, 0.45833333, 0.47222222,
       0.48611111, 0.5       , 0.51388889, 0.52777778, 0.54166667,
       0.55555556, 0.56944444, 0.58333333, 0.59722222, 0.61111111,
       0.625     , 0.63888889, 0.65277778, 0.66666667, 0.68055556,
       0.69444444, 0.70833333, 0.72222222, 0.73611111, 0.75      ,
       0.76388889, 0.77777778, 0.79166667, 0.80555556, 0.81944444,
       0.83333333, 0.84722222, 0.86111111, 0.875     , 0.88888889,
       0.90277778, 0.91666667, 0.93055556, 0.94444444, 0.95833333,
       0.97222222, 0.98611111, 1.        ])

In [112]:
# Double-check mapping again and overwrite
international_matches['Result'] = international_matches['Result'].map({'Home': 1, 'Away': 0})
print(international_matches['Result'].unique())
print(international_matches['Result'].dtype)

[1 0]
int64


In [113]:
features = international_matches[['Home Team', 'Away Team', 'Home Win Rate', 'Away Win Rate', 'Home ELO', 'Away ELO', 'Tournament Weight', 'Recent Weight']]
target = international_matches['Result']
Tweights = international_matches['Tournament Weight']
Rweights = international_matches['Recent Weight']

print("Features shape:", features.shape)
print("Target shape:", target.shape)
print("Any NaNs in features?", features.isnull().values.any())
print("Any NaNs in target?", target.isnull().values.any())

Features shape: (10640, 8)
Target shape: (10640,)
Any NaNs in features? False
Any NaNs in target? False


In [114]:
international_matches['Result'].unique()
international_matches['Home Stadium'].unique()
result_mapping = {'Home': 1, 'Away': 0}
international_matches['Result'] = international_matches['Result'].map(result_mapping)

In [115]:
# Confirm the mapping worked
print(international_matches['Result'].unique())

[nan]


In [116]:
international_matches.head()

,ID,Tournament,Home Team,Home Goals,Away Goals,Away Team,Winning Team,Losing Team,Home Stadium,Year,Result,Home Team Name,Away Team Name,Home ELO,Away ELO,Home Win Rate,Away Win Rate,Tournament Weight,Recent Weight
1810,1811,Friendly,41,4,1,152,Croatia,Serbia,True,1950,NaN,Croatia,Serbia,1500.0,1500.0,0.5,0.5,0.3,0.0
1811,1812,Friendly,86,3,1,16,Italy,Belgium,True,1950,NaN,Italy,Belgium,1500.0,1500.0,0.5,0.5,0.3,0.0
1815,1816,Copa Chevallier Boutell,6,4,0,134,Argentina,Paraguay,True,1950,NaN,Argentina,Paraguay,1500.0,1500.0,0.5,0.5,0.6,0.0
1816,1817,FIFA World Cup qualification,161,5,1,138,Spain,Portugal,True,1950,NaN,Spain,Portugal,1500.0,1500.0,0.5,0.5,0.6,0.0
1817,1818,Friendly,34,1,5,182,Uruguay,Chile,True,1950,NaN,Chile,Uruguay,1500.0,1500.0,0.5,0.5,0.3,0.0


In [117]:
international_matches.dtypes

,0
ID,int64
Tournament,object
Home Team,int64
Home Goals,int64
Away Goals,int64
Away Team,int64
Winning Team,object
Losing Team,object
Home Stadium,bool
Year,int64


In [125]:
weights = Tweights * Rweights
def objective(trial):
    n_estimators = trial.suggest_int('n_estimators', 100, 500)
    max_depth = trial.suggest_int('max_depth', 5, 50)
    min_samples_split = trial.suggest_int('min_samples_split', 2, 10)

    model = RandomForestClassifier(
        n_estimators=n_estimators,
        max_depth=max_depth,
        min_samples_split=min_samples_split,
        random_state=9
    )

    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=9)
    accuracies = []

    for train_idx, val_idx in skf.split(features, target):
        X_train_cv, X_val_cv = features.iloc[train_idx], features.iloc[val_idx]
        y_train_cv, y_val_cv = target.iloc[train_idx], target.iloc[val_idx]
        w_train_cv = weights.iloc[train_idx]

        model.fit(X_train_cv, y_train_cv, sample_weight=w_train_cv)
        acc = model.score(X_val_cv, y_val_cv)
        accuracies.append(acc)

    return np.mean(accuracies)
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=30)

print("Best Hyperparameters:", study.best_params)

[I 2025-07-03 18:58:26,271] A new study created in memory with name: no-name-58d849a8-eec0-4e1e-bab3-0e1d3264fc97
[I 2025-07-03 18:58:42,881] Trial 0 finished with value: 0.749154135338346 and parameters: {'n_estimators': 499, 'max_depth': 6, 'min_samples_split': 2}. Best is trial 0 with value: 0.749154135338346.
[I 2025-07-03 18:58:49,943] Trial 1 finished with value: 0.7487781954887218 and parameters: {'n_estimators': 105, 'max_depth': 50, 'min_samples_split': 6}. Best is trial 0 with value: 0.749154135338346.
[I 2025-07-03 18:59:23,380] Trial 2 finished with value: 0.7492481203007518 and parameters: {'n_estimators': 476, 'max_depth': 28, 'min_samples_split': 6}. Best is trial 2 with value: 0.7492481203007518.
[I 2025-07-03 18:59:37,594] Trial 3 finished with value: 0.7476503759398495 and parameters: {'n_estimators': 202, 'max_depth': 20, 'min_samples_split': 5}. Best is trial 2 with value: 0.7492481203007518.
[I 2025-07-03 18:59:50,469] Trial 4 finished with value: 0.749624060150376

Best Hyperparameters: {'n_estimators': 255, 'max_depth': 11, 'min_samples_split': 9}


In [127]:
X_train, X_test, y_train, y_test, w_train, w_test = train_test_split(
    features, target, weights,
    test_size=0.2, random_state=9, stratify=target
)
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
accuracies = []

for train_idx, val_idx in skf.split(X_train, y_train):
    X_train_cv, X_val_cv = X_train.iloc[train_idx], X_train.iloc[val_idx]
    y_train_cv, y_val_cv = y_train.iloc[train_idx], y_train.iloc[val_idx]
    w_train_cv = w_train.iloc[train_idx]  # sample weights for training fold

    clf_cv = RandomForestClassifier(**study.best_params, random_state=9)
    clf_cv.fit(X_train_cv, y_train_cv, sample_weight=w_train_cv)

    preds = clf_cv.predict(X_val_cv)
    acc = accuracy_score(y_val_cv, preds)
    accuracies.append(acc)

print("Manual CV Accuracy:", np.mean(accuracies))

Manual CV Accuracy: 0.7496464040440143


In [ ]:
team_encoder = dict(zip(le.classes_, le.transform(le.classes_)))

TEAM_DATA = {}

for team_name in team_encoder:
    encoded = team_encoder[team_name]

    home_matches = international_matches[international_matches['Home Team'] == encoded]
    away_matches = international_matches[international_matches['Away Team'] == encoded]
    all_matches = pd.concat([home_matches, away_matches]).sort_values(by='Year')

    if not all_matches.empty:
        last_home = home_matches['Home ELO'].iloc[-1] if not home_matches.empty else 1500
        last_away = away_matches['Away ELO'].iloc[-1] if not away_matches.empty else 1500
        elo = (last_home + last_away) / 2
    else:
        elo = 1500

    if not all_matches.empty:
        home_wr = home_matches['Home Win Rate'].iloc[-1] if not home_matches.empty else 0.5
        away_wr = away_matches['Away Win Rate'].iloc[-1] if not away_matches.empty else 0.5
        win_rate = (home_wr + away_wr) / 2
    else:
        win_rate = 0.5

    TEAM_DATA[team_name] = {
        "encoded": encoded,
        "elo": elo,
        "win_rate": win_rate
    }

print("Loaded teams:", list(TEAM_DATA.keys()))

def generate_match_features(team1: str, team2: str) -> pd.DataFrame:
    t1 = TEAM_DATA[team1]
    t2 = TEAM_DATA[team2]

    return pd.DataFrame([{
        'Home Team': t1["encoded"],
        'Away Team': t2["encoded"],
        'Home Win Rate': t1["win_rate"],
        'Away Win Rate': t2["win_rate"],
        'Home ELO': t1["elo"],
        'Away ELO': t2["elo"],
        'Tournament Weight': 1.0,
        'Recent Weight': 1.0
    }])

def predict_match(team1, team2, model, n=1000):
    features = generate_match_features(team1, team2)
    probs = model.predict_proba(features)[0]

    outcomes = np.random.choice(
        ['Team2 Win', 'Team1 Win'], size=n, p=[probs[0], probs[1]]
    )
    team1_wins = np.count_nonzero(outcomes == 'Team1 Win')
    team2_wins = n - team1_wins

    if team1_wins > team2_wins:
        outcome = "Team1 Win"
        winner = team1
        confidence = (team1_wins / n) * 100
    else:
        outcome = "Team2 Win"
        winner = team2
        confidence = (team2_wins / n) * 100

    return outcome, winner, confidence, probs

model = clf_cv

GROUPS = {
    "A": ["Qatar", "Ecuador", "Senegal", "Netherlands"],
    "B": ["England", "Iran", "United States", "Wales"],
    "C": ["Argentina", "Saudi Arabia", "Mexico", "Poland"],
    "D": ["France", "Australia", "Denmark", "Tunisia"],
    "E": ["Spain", "Costa Rica", "Germany", "Japan"],
    "F": ["Belgium", "Canada", "Morocco", "Croatia"],
    "G": ["Brazil", "Serbia", "Switzerland", "Cameroon"],
    "H": ["Portugal", "Ghana", "Uruguay", "Egypt"]
}

GROUP_MATCHES = {
    "A": [("Qatar", "Ecuador"), ("Senegal", "Netherlands"), ("Qatar", "Senegal"),
           ("Netherlands", "Ecuador"), ("Netherlands", "Qatar"), ("Ecuador", "Senegal")],
    "B": [("England", "Iran"), ("United States", "Wales"), ("Wales", "Iran"),
           ("England", "United States"), ("Wales", "England"), ("Iran", "United States")],
    "C": [("Argentina", "Saudi Arabia"), ("Mexico", "Poland"), ("Poland", "Saudi Arabia"),
           ("Argentina", "Mexico"), ("Poland", "Argentina"), ("Saudi Arabia", "Mexico")],
    "D": [("Denmark", "Tunisia"), ("France", "Australia"), ("Tunisia", "Australia"),
           ("France", "Denmark"), ("Australia", "Denmark"), ("Tunisia", "France")],
    "E": [("Germany", "Japan"), ("Spain", "Costa Rica"), ("Japan", "Costa Rica"),
           ("Spain", "Germany"), ("Japan", "Spain"), ("Costa Rica", "Germany")],
    "F": [("Morocco", "Croatia"), ("Belgium", "Canada"), ("Belgium", "Morocco"),
           ("Croatia", "Canada"), ("Croatia", "Belgium"), ("Canada", "Morocco")],
    "G": [("Switzerland", "Cameroon"), ("Brazil", "Serbia"), ("Cameroon", "Serbia"),
           ("Brazil", "Switzerland"), ("Serbia", "Switzerland"), ("Cameroon", "Brazil")],
    "H": [("Uruguay", "Egypt"), ("Portugal", "Ghana"), ("Egypt", "Ghana"),
           ("Portugal", "Uruguay"), ("Ghana", "Uruguay"), ("Egypt", "Portugal")]
}

def simulate_groups(model):
    standings = {}
    for grp, teams in GROUPS.items():
        stats = {t: {'pts': 0, 'conf_for': 0.0} for t in teams}
        for t1, t2 in GROUP_MATCHES[grp]:
            outcome, winner, conf, probs = predict_match(t1, t2, model)
            if outcome == "Team1 Win":
                stats[t1]['pts'] += 3
                stats[t1]['conf_for'] += conf
            elif outcome == "Team2 Win":
                stats[t2]['pts'] += 3
                stats[t2]['conf_for'] += conf
            else:
                stats[t1]['pts'] += 1
                stats[t2]['pts'] += 1
                stats[t1]['conf_for'] += probs[1] * 50
                stats[t2]['conf_for'] += probs[1] * 50

        ordered = sorted(stats.items(), key=lambda kv: (kv[1]['pts'], kv[1]['conf_for'], np.random.random()), reverse=True)
        standings[grp] = ordered
    return standings

def round_of_16_pairs(standings):
    gw = lambda g: standings[g][0][0]
    gr = lambda g: standings[g][1][0]
    return [
        (gw("A"), gr("B")), (gw("C"), gr("D")), (gw("E"), gr("F")), (gw("G"), gr("H")),
        (gw("B"), gr("A")), (gw("D"), gr("C")), (gw("F"), gr("E")), (gw("H"), gr("G"))
    ]

def simulate_knockout_pairs(pairs, model):
    winners, log = [], []
    for t1, t2 in pairs:
        outcome, winner, conf, _ = predict_match(t1, t2, model)
        winners.append(winner)
        log.append(f"{t1} vs {t2}  →  {winner}  ({conf:.1f}% confident)")
    return winners, log

def simulate_world_cup(model):
    standings = simulate_groups(model)
    r16 = round_of_16_pairs(standings)
    qf_teams, r16_log = simulate_knockout_pairs(r16, model)
    qf = [(qf_teams[0], qf_teams[1]), (qf_teams[2], qf_teams[3]),
          (qf_teams[4], qf_teams[5]), (qf_teams[6], qf_teams[7])]
    sf_teams, qf_log = simulate_knockout_pairs(qf, model)
    sf = [(sf_teams[0], sf_teams[1]), (sf_teams[2], sf_teams[3])]
    finalists, sf_log = simulate_knockout_pairs(sf, model)
    final_pair = [(finalists[0], finalists[1])]
    [champion], final_log = simulate_knockout_pairs(final_pair, model)
    final_conf = float(final_log[0].split('(')[-1].rstrip('% confident)'))
    return {
        "standings": standings,
        "r16_log": r16_log,
        "qf_log": qf_log,
        "sf_log": sf_log,
        "final_log": final_log,
        "champion": champion,
        "champion_confidence": final_conf
    }

if __name__ == "__main__":
    missing = [team for group in GROUPS.values() for team in group if team not in TEAM_DATA]
    if missing:
        print("\n\u274C Missing teams in TEAM_DATA:", missing)
    else:
        results = simulate_world_cup(model)
        print("\n=== GROUP WINNERS & RUNNERS‑UP ===")
        for g in "ABCDEFGH":
            first, second = results["standings"][g][0][0], results["standings"][g][1][0]
            print(f"Group {g}: {first} (1st)  –  {second} (2nd)")

        print("\n=== ROUND OF 16 ===")
        for line in results["r16_log"]:
            print(line)

        print("\n=== QUARTER‑FINALS ===")
        for line in results["qf_log"]:
            print(line)

        print("\n=== SEMI‑FINALS ===")
        for line in results["sf_log"]:
            print(line)

        print("\n=== FINAL ===")
        print(results["final_log"][0])

        print(f"\n🏆  CHAMPION: {results['champion']} with {results['champion_confidence']:.1f}% confidence\n")

✅ Loaded teams: ['Afghanistan', 'Albania', 'Algeria', 'Andorra', 'Angola', 'Antigua and Barbuda', 'Argentina', 'Armenia', 'Aruba', 'Australia', 'Austria', 'Azerbaijan', 'Bahrain', 'Bangladesh', 'Barbados', 'Belarus', 'Belgium', 'Belize', 'Benin', 'Bermuda', 'Bhutan', 'Bolivia', 'Bosnia and Herzegovina', 'Botswana', 'Brazil', 'Bulgaria', 'Burkina Faso', 'Burundi', 'Cambodia', 'Cameroon', 'Canada', 'Cayman Islands', 'Central African Republic', 'Chad', 'Chile', 'China PR', 'Colombia', 'Comoros', 'Congo', 'Cook Islands', 'Costa Rica', 'Croatia', 'Cuba', 'Curaçao', 'Cyprus', 'Czech Republic', 'Denmark', 'Djibouti', 'Dominica', 'Dominican Republic', 'Ecuador', 'Egypt', 'El Salvador', 'England', 'Equatorial Guinea', 'Eritrea', 'Estonia', 'Eswatini', 'Ethiopia', 'Faroe Islands', 'Fiji', 'Finland', 'France', 'Gabon', 'Gambia', 'Georgia', 'Germany', 'Ghana', 'Gibraltar', 'Greece', 'Grenada', 'Guam', 'Guatemala', 'Guinea', 'Guinea-Bissau', 'Guyana', 'Haiti', 'Honduras', 'Hong Kong', 'Hungary', 'I